# Projection of d onto $\text{div }=0$ manifold
I nee to solve the minimization problem:
$$ 
\text{min}_{d\in H(\text{div})} \| d - \hat{d} \|_{L_2} \\
\text{div}(d) = 0
$$
Using lagrange procedure we find the following problem: Find $(d,q)\in H(div)\times L_2$ s.t.
$$
\begin{align}
(d, p)  + (\text{div} d, p) & = (\hat{d}, p)\quad \forall q \in L_2 \\
(\text{div} v, q)&=0 \quad \forall v \in H(\text{div})
\end{align}
$$
Obviously the discretization implies that the order of $L_2$ elemens are one less then the other space:

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube

n = specialcf.normal(3)

Maxh = 0.15
Order = 3
mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))


fesDiv = HDiv(mesh, order=Order  , dirichlet= ".*")
fesL2 = L2(mesh, order=Order-1)#, dirichlet= ".*")


fesP =  fesDiv*fesL2 
peak = exp(-25*( (x-0.5)**2 + (y-0.5)**2 +(z-0.5)**2))
PEAK = CF ( (peak, peak, peak ), dims=(3,1))
eps = CF((1), dims=(1,1))
mu =  CF((1), dims=(1,1))
elfiel2 = GridFunction(fesP) 
d, q = elfiel2.components
d.Set ( PEAK, bonus_intorder=10, dual=True)

Pelfield = GridFunction(fesP) 
Pd, Pq = Pelfield.components


# We need to define a projection 

u, p  = fesP.TrialFunction()
v, q  = fesP.TestFunction()

P = BilinearForm(fesP)
P +=  v*u * dx
P +=  div( v)*p * dx
P +=  div( u)*q * dx

F = BilinearForm(fesP)
F +=  v*u * dx




P.Assemble()
F.Assemble()

invP = P.mat.Inverse()#fesP.FreeDofs(True),"pardiso")

Pelfield.vec.data =invP*(F.mat*elfiel2.vec) 


print("d")
Draw(d, mesh, clipping=(1,1,1))
print("div d")
Draw(div(d), mesh, clipping=(1,1,1), draw_surf=False)


print("Pd")
Draw(Pd, mesh, clipping=(1,1,1))
print("div Pd")
Draw(div(Pd), mesh, clipping=(1,1,1), draw_surf=False)



d


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

div d


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

Pd


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

div Pd


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

BaseWebGuiScene